In [36]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import unidecode
import os

In [37]:
soup = BeautifulSoup(open("../datasets/source_code_mutual_art.html"))
soup2 = BeautifulSoup(open("../datasets/artistas_seguidos_1.txt").read())
soup3 = BeautifulSoup(open("../datasets/artistas_seguidos_2.txt").read())
soup4 = BeautifulSoup(open("../datasets/artistas_seguidos_3.txt").read())
soup5 = BeautifulSoup(open("../datasets/artistas_seguidos_5.txt").read())
soup6 = BeautifulSoup(open("../datasets/artistas_seguidos_6.txt").read())
soup7 = BeautifulSoup(open("../datasets/artistas_seguidos_7.txt").read())
soup8 = BeautifulSoup(open("../datasets/paises_latam_1.txt").read())
soup9 = BeautifulSoup(open("../datasets/paises_latam_2.txt").read())
soup10 = BeautifulSoup(open("../datasets/paises_latam_3.txt").read())

soups = [soup,soup2,soup3,soup4,soup5,soup6,soup7,soup8,soup9,soup10]

In [176]:
precios_fecha=[['precio_estimado','precio_venta','fecha','autor','casa_subasta','tipo','tecnica','dimensiones','fecha_creacion','titulo','precio_ajustado']]
for soup in soups:
    obras = soup.findAll("div",{"class":"item table-row row v2__page"})
    for obra in obras:
        features=obra.findAll(lambda tag: tag.name=='div' and tag.get("class")==['v2__artwork-detail__price-col'])
        datos_obra=obra.findAll(lambda tag: tag.get("class")==['v2__link--primary'])
        tecnica = obra.findAll("span", {"itemprop": "artMedium"})
        dimensiones = obra.findAll("span", {"itemprop": "artDimensions"})
        fecha_creacion = obra.findAll("span", {"itemprop": "dateCreated"})
        titulo = obra.find("a", {"href": re.compile(r"/Artwork/.*")})
        if (len(features)==3 and len(datos_obra)==3 and len(tecnica)==1 and len(dimensiones)>=1):
            if (len(fecha_creacion) > 0):
                fecha_pintura = fecha_creacion[0].text
            else: 
                fecha_pintura = None
            precio_ajustado=re.findall(r"([0-9]+,?([0-9]+) [A-Z]+)",str(features[1]))
            if (len(precio_ajustado)>1):
                if (type(precio_ajustado[len(precio_ajustado)-2])==tuple):
                    precio_ajustado = precio_ajustado[len(precio_ajustado)-2][0]
                else:
                    precio_ajustado = precio_ajustado[len(precio_ajustado)-2]
            else: 
                precio_ajustado = None
            precio_estimado=features[0].text
            precio_venta=features[1].text
            fecha=features[2].text
            autor=datos_obra[0].text
            casa_subasta=datos_obra[1].text
            tipo=datos_obra[2].text
            tecnica=tecnica[0].text
            dimensiones=dimensiones[0].text
            titulo = titulo.find("img")['title']
            precios_fecha.append([precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_pintura,titulo,precio_ajustado])
        else:
            pass

In [177]:
len(precios_fecha)

22298

In [178]:
data=pd.DataFrame(precios_fecha[1:],columns=precios_fecha[0])
for column in data.columns:
    data[str(column)]=data[str(column)].replace(r'\s+', ' ', regex=True)
    data[str(column)]=data[unidecode.unidecode(str(column))]
data.drop_duplicates(inplace=True)

In [179]:
data.shape

(20128, 11)

In [186]:
data[data.precio_ajustado.isna() != True].shape

(13646, 11)

In [182]:
data[(data.precio_venta != ' Upcoming ') & (data.precio_venta != ' Live Now ') & (data.precio_venta != ' Results Coming Soon ') & (data.precio_venta != ' Not Sold ')]

,precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_creacion,titulo,precio_ajustado
862,700 EUR,700 EUR,"Sep 21, 2022",Jerson Jimenez,"Dorotheum, Vienna",Charity Auction of the Salvatorians,Oil on canvas,50 x 40 cm,2021,"""Experiencia"" by Jerson Jimenez, 2021",705 USD
975,"4,000 - 4,400 EUR","5,120 EUR","Sep 20, 2022",Julio Le Parc,"Dorotheum, Vienna",Modern and Contemporary Art,screenprint on cardboard,"41 x 41 x 5 cm, 8, 1970,",1970,"Relief 8 by Julio Le Parc, 1970","5,154 USD"
976,"1,000 - 1,500 GBP","1,275 GBP","Sep 20, 2022",Raúl Anguiano,Bonhams Knightsbridge,Travel & Exploration,oil on canvas,65.1 x 50.2cm (25 x 19 3/4in),1952,"Portrait of a young woman by Raúl Anguiano, 1952","1,485 USD"
977,"2,000 - 3,000 AUD","1,200 AUD*","Sep 20, 2022",Jose Haro,Ozbid Auctions,HUGE MODERN ART AUCTION - LIVE ONLINE,Mixed media,94cm x 153cm,None,Macaw by Jose Haro,824 USD
978,350 - 400 CHF,225 CHF*,"Sep 20, 2022",Jorge Camacho,TGP Auction,Pop & Contemporary Art,lithograph on Guarro paper,56 x 76 cm 56 x 76 cm 10 H. 10 H.,1974,"Cenizas by Jorge Camacho, 1974",231 USD
...,...,...,...,...,...,...,...,...,...,...,...
22288,"800 - 1,200 GBP",625 GBP,"Mar 14, 2009",Carolina Raquel Antich,Phillips London,Saturday@Phillips,Acrylic on canvas.,90 x 70 cm. (35 ½ x 27 ½ in.),2005,"Comparsa (Lei) by Carolina Raquel Antich, 2005","1,204 USD"
22289,"1,000 - 1,500 GBP",313 GBP,"Mar 14, 2009",Carolina Raquel Antich,Phillips London,Saturday@Phillips,Acrylic on canvas.,106 x 85 cm. (41 3/4 x 33 1/2 in.),2005,"Dipingere by Carolina Raquel Antich, 2005",603 USD
22291,100 - 150 GBP,100 GBP,"Mar 25, 2009",Axel Amuchastegui,Woolley & Wallis,"Oil paintings, Watercolours, Prints & Miniatur...",Watercolour heightened with bodycolour,25.5 x 21.5cm; 10 x 8½in,None,Racoon by Axel Amuchastegui,202 USD
22292,"6,500 - 7,000 EUR","7,500 EUR","Mar 24, 2009 - Mar 26, 2009",Adriano Spilimbergo,"Il Ponte Auction House, Via Pontaccio","Furnishing, Old Master Painting of the 19th Ce...",oil on canvas,cm 50x60,1963,"Venezia - San Giorgio by Adriano Spilimbergo, ...","13,994 USD"


In [187]:
data.to_csv('../datasets/data_mutualart_completo.csv')